In [1]:
# imports
import pandas as pd
import numpy as np
from sklearn.compose import make_column_transformer
from sklearn import preprocessing
from datetime import datetime, timedelta

In [2]:
persons = pd.read_csv('../data/Motor_Vehicle_Collisions_-_Person_20241103.csv', low_memory=False)

In [3]:
persons

,UNIQUE_ID,COLLISION_ID,CRASH_DATE,CRASH_TIME,PERSON_ID,PERSON_TYPE,PERSON_INJURY,VEHICLE_ID,PERSON_AGE,EJECTION,...,BODILY_INJURY,POSITION_IN_VEHICLE,SAFETY_EQUIPMENT,PED_LOCATION,PED_ACTION,COMPLAINT,PED_ROLE,CONTRIBUTING_FACTOR_1,CONTRIBUTING_FACTOR_2,PERSON_SEX
0,10249006,4229554,10/26/2019,9:43,31aa2bc0-f545-444f-8cdb-f1cb5cf00b89,Occupant,Unspecified,19141108.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Registrant,NaN,NaN,U
1,10255054,4230587,10/25/2019,15:15,4629e500-a73e-48dc-b8fb-53124d124b80,Occupant,Unspecified,19144075.0,33.0,Not Ejected,...,Does Not Apply,"Front passenger, if two or more persons, inclu...",Lap Belt & Harness,NaN,NaN,Does Not Apply,Passenger,NaN,NaN,F
2,10253177,4230550,10/26/2019,17:55,ae48c136-1383-45db-83f4-2a5eecfb7cff,Occupant,Unspecified,19143133.0,55.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Registrant,NaN,NaN,M
3,6650180,3565527,11/21/2016,13:05,2782525,Occupant,Unspecified,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Notified Person,NaN,NaN,NaN
4,10255516,4231168,10/25/2019,11:16,e038e18f-40fb-4471-99cf-345eae36e064,Occupant,Unspecified,19144329.0,7.0,Not Ejected,...,Does Not Apply,Right rear passenger or motorcycle sidecar pas...,Lap Belt,NaN,NaN,Does Not Apply,Passenger,NaN,NaN,F
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5513744,13193398,4767904,10/29/2024,15:32,1b635a58-c685-4635-9db6-d1ee03aa42b9,Occupant,Injured,20774759.0,71.0,Not Ejected,...,Neck,"Front passenger, if two or more persons, inclu...",Lap Belt & Harness,NaN,NaN,Complaint of Pain or Nausea,Passenger,NaN,NaN,F
5513745,13193338,4767864,10/29/2024,17:00,a9ecd00b-c096-4769-82f9-20010a519c59,Occupant,Unspecified,20774731.0,49.0,Not Ejected,...,Does Not Apply,Driver,Lap Belt & Harness,NaN,NaN,Does Not Apply,Driver,NaN,NaN,F
5513746,13192746,4767688,10/28/2024,0:36,be95a1de-a8b2-4b9b-a700-add351c14225,Occupant,Unspecified,20774392.0,69.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Registrant,NaN,NaN,M
5513747,13192873,4767781,10/24/2024,16:10,60730576-0035-437b-8508-1db0b53afb46,Occupant,Unspecified,20774464.0,35.0,Not Ejected,...,Does Not Apply,"Front passenger, if two or more persons, inclu...",Lap Belt & Harness,NaN,NaN,Does Not Apply,Passenger,NaN,NaN,F


In [4]:
cutoff_date = datetime.now() - timedelta(days=10*365)
persons['CRASH_DATE'] = pd.to_datetime(persons['CRASH_DATE'], format='%m/%d/%Y')
filtered_persons = persons[persons['CRASH_DATE'] >= cutoff_date]
filtered_persons.iloc[0:4]
filtered_persons.shape

(5381702, 21)

In [5]:
filtered_persons.dropna(subset=['PERSON_AGE'], inplace=True)
filtered_persons.iloc[0:4]
filtered_persons.shape

C:\Users\Johann Ruiz\AppData\Local\Temp\ipykernel_29332\1922748913.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_persons.dropna(subset=['PERSON_AGE'], inplace=True)


(4781507, 21)

In [6]:
filtered_persons.dropna(subset=['PERSON_SEX'], inplace=True)
filtered_persons.iloc[0:4]
filtered_persons.shape

C:\Users\Johann Ruiz\AppData\Local\Temp\ipykernel_29332\210794637.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_persons.dropna(subset=['PERSON_SEX'], inplace=True)


(4581723, 21)

In [7]:
filtered_persons.iloc[1]

UNIQUE_ID                                            10253177
COLLISION_ID                                          4230550
CRASH_DATE                                2019-10-26 00:00:00
CRASH_TIME                                              17:55
PERSON_ID                ae48c136-1383-45db-83f4-2a5eecfb7cff
PERSON_TYPE                                          Occupant
PERSON_INJURY                                     Unspecified
VEHICLE_ID                                         19143133.0
PERSON_AGE                                               55.0
EJECTION                                                  NaN
EMOTIONAL_STATUS                                          NaN
BODILY_INJURY                                             NaN
POSITION_IN_VEHICLE                                       NaN
SAFETY_EQUIPMENT                                          NaN
PED_LOCATION                                              NaN
PED_ACTION                                                NaN
COMPLAIN

In [8]:
filtered_persons["CRASH_DATE_YEAR"] = filtered_persons["CRASH_DATE"].dt.year
filtered_persons["CRASH_DATE_MONTH"] = filtered_persons["CRASH_DATE"].dt.month
filtered_persons["CRASH_DATE_DAY"] = filtered_persons["CRASH_DATE"].dt.day
filtered_persons.drop(['CRASH_DATE'], axis=1, inplace=True)

C:\Users\Johann Ruiz\AppData\Local\Temp\ipykernel_29332\1836540385.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_persons["CRASH_DATE_YEAR"] = filtered_persons["CRASH_DATE"].dt.year
C:\Users\Johann Ruiz\AppData\Local\Temp\ipykernel_29332\1836540385.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_persons["CRASH_DATE_MONTH"] = filtered_persons["CRASH_DATE"].dt.month
C:\Users\Johann Ruiz\AppData\Local\Temp\ipykernel_29332\1836540385.py:3: SettingWithCopyWarning: 
A value is trying to

In [9]:
filtered_persons.drop(['UNIQUE_ID', 'PERSON_ID'], axis=1, inplace=True)

C:\Users\Johann Ruiz\AppData\Local\Temp\ipykernel_29332\771773966.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_persons.drop(['UNIQUE_ID', 'PERSON_ID'], axis=1, inplace=True)


Focus on Car to Car collisions (Remove bicyclist, anything to do with a bike)

Create column of amount of people involved in accident (OWN VECHICLE VS OTHER VEHICLE)

In [10]:
print(filtered_persons["PED_ROLE"].unique())
filtered_persons = filtered_persons.loc[filtered_persons["PED_ROLE"].isin(['Driver', 'Passenger'])]
filtered_persons

['Passenger' 'Registrant' 'Driver' 'Pedestrian' 'Other' 'In-Line Skater'
 'Policy Holder' 'Witness' 'Owner']


,COLLISION_ID,CRASH_TIME,PERSON_TYPE,PERSON_INJURY,VEHICLE_ID,PERSON_AGE,EJECTION,EMOTIONAL_STATUS,BODILY_INJURY,POSITION_IN_VEHICLE,...,PED_LOCATION,PED_ACTION,COMPLAINT,PED_ROLE,CONTRIBUTING_FACTOR_1,CONTRIBUTING_FACTOR_2,PERSON_SEX,CRASH_DATE_YEAR,CRASH_DATE_MONTH,CRASH_DATE_DAY
1,4230587,15:15,Occupant,Unspecified,19144075.0,33.0,Not Ejected,Does Not Apply,Does Not Apply,"Front passenger, if two or more persons, inclu...",...,NaN,NaN,Does Not Apply,Passenger,NaN,NaN,F,2019,10,25
4,4231168,11:16,Occupant,Unspecified,19144329.0,7.0,Not Ejected,Does Not Apply,Does Not Apply,Right rear passenger or motorcycle sidecar pas...,...,NaN,NaN,Does Not Apply,Passenger,NaN,NaN,F,2019,10,25
5,4230743,19:15,Occupant,Injured,19143343.0,27.0,Not Ejected,Conscious,Back,Driver,...,NaN,NaN,Complaint of Pain or Nausea,Driver,NaN,NaN,M,2019,10,24
8,4229808,13:04,Occupant,Unspecified,19141630.0,36.0,Not Ejected,Does Not Apply,Does Not Apply,Driver,...,NaN,NaN,Does Not Apply,Driver,NaN,NaN,M,2019,10,26
10,4230737,0:41,Occupant,Unspecified,19143334.0,30.0,Not Ejected,Does Not Apply,Does Not Apply,Driver,...,NaN,NaN,Does Not Apply,Driver,NaN,NaN,M,2019,10,24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5513741,4767880,14:40,Occupant,Injured,20774505.0,49.0,Not Ejected,Conscious,Knee-Lower Leg Foot,Unknown,...,NaN,NaN,Complaint of Pain or Nausea,Passenger,NaN,NaN,F,2024,10,28
5513744,4767904,15:32,Occupant,Injured,20774759.0,71.0,Not Ejected,Unknown,Neck,"Front passenger, if two or more persons, inclu...",...,NaN,NaN,Complaint of Pain or Nausea,Passenger,NaN,NaN,F,2024,10,29
5513745,4767864,17:00,Occupant,Unspecified,20774731.0,49.0,Not Ejected,Does Not Apply,Does Not Apply,Driver,...,NaN,NaN,Does Not Apply,Driver,NaN,NaN,F,2024,10,29
5513747,4767781,16:10,Occupant,Unspecified,20774464.0,35.0,Not Ejected,Does Not Apply,Does Not Apply,"Front passenger, if two or more persons, inclu...",...,NaN,NaN,Does Not Apply,Passenger,NaN,NaN,F,2024,10,24


In [11]:
print(filtered_persons["PED_ROLE"].unique())
print(filtered_persons["POSITION_IN_VEHICLE"].unique())

['Passenger' 'Driver']
['Front passenger, if two or more persons, including the driver, are in the front seat'
 'Right rear passenger or motorcycle sidecar passenger' 'Driver'
 'Left rear passenger, or rear passenger on a bicycle, motorcycle, snowmobile'
 'Middle front seat, or passenger lying across a seat' 'Unknown'
 'Middle rear seat, or passenger lying across a seat'
 'Any person in the rear of a station wagon, pick-up truck, all passengers on a bus, etc'
 'Riding/Hanging on Outside'
 'If one person is seated on another person&apos;s lap' 'Does Not Apply'
 nan]


In [12]:
drivers1 = filtered_persons.loc[filtered_persons["PED_ROLE"] == 'Driver']
drivers = drivers1.loc[filtered_persons["POSITION_IN_VEHICLE"] == 'Driver']
drivers

,COLLISION_ID,CRASH_TIME,PERSON_TYPE,PERSON_INJURY,VEHICLE_ID,PERSON_AGE,EJECTION,EMOTIONAL_STATUS,BODILY_INJURY,POSITION_IN_VEHICLE,...,PED_LOCATION,PED_ACTION,COMPLAINT,PED_ROLE,CONTRIBUTING_FACTOR_1,CONTRIBUTING_FACTOR_2,PERSON_SEX,CRASH_DATE_YEAR,CRASH_DATE_MONTH,CRASH_DATE_DAY
5,4230743,19:15,Occupant,Injured,19143343.0,27.0,Not Ejected,Conscious,Back,Driver,...,NaN,NaN,Complaint of Pain or Nausea,Driver,NaN,NaN,M,2019,10,24
8,4229808,13:04,Occupant,Unspecified,19141630.0,36.0,Not Ejected,Does Not Apply,Does Not Apply,Driver,...,NaN,NaN,Does Not Apply,Driver,NaN,NaN,M,2019,10,26
10,4230737,0:41,Occupant,Unspecified,19143334.0,30.0,Not Ejected,Does Not Apply,Does Not Apply,Driver,...,NaN,NaN,Does Not Apply,Driver,NaN,NaN,M,2019,10,24
18,4230715,8:50,Bicyclist,Injured,19143832.0,42.0,Not Ejected,Unknown,Knee-Lower Leg Foot,Driver,...,NaN,NaN,None Visible,Driver,NaN,NaN,M,2019,10,26
19,4230612,16:12,Occupant,Unspecified,19143548.0,55.0,Not Ejected,Does Not Apply,Does Not Apply,Driver,...,NaN,NaN,Does Not Apply,Driver,NaN,NaN,M,2019,10,24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5513730,4767496,13:52,Other Motorized,Unspecified,20774121.0,13.0,Not Ejected,Does Not Apply,Does Not Apply,Driver,...,NaN,NaN,Does Not Apply,Driver,NaN,NaN,M,2024,10,29
5513732,4767506,15:45,Occupant,Unspecified,20773902.0,34.0,Not Ejected,Does Not Apply,Does Not Apply,Driver,...,NaN,NaN,Does Not Apply,Driver,NaN,NaN,M,2024,10,29
5513733,4767480,15:15,Occupant,Unspecified,20774286.0,23.0,Not Ejected,Does Not Apply,Does Not Apply,Driver,...,NaN,NaN,Does Not Apply,Driver,NaN,NaN,M,2024,10,29
5513735,4767575,13:30,Occupant,Unspecified,20774084.0,66.0,Not Ejected,Does Not Apply,Does Not Apply,Driver,...,NaN,NaN,Does Not Apply,Driver,NaN,NaN,M,2024,10,29


In [13]:
inVehiclePeople = filtered_persons.copy()

In [14]:
print(drivers["PERSON_TYPE"].unique())
print(inVehiclePeople["PERSON_TYPE"].unique())

['Occupant' 'Bicyclist' 'Other Motorized']
['Occupant' 'Bicyclist' 'Other Motorized']


In [15]:
drivers = drivers.loc[drivers["PERSON_TYPE"] != 'Bicyclist']
drivers = drivers.loc[drivers["PERSON_TYPE"] != 'Other Motorized']
inVehiclePeople = inVehiclePeople.loc[inVehiclePeople["PERSON_TYPE"] != 'Bicyclist']
inVehiclePeople = inVehiclePeople.loc[inVehiclePeople["PERSON_TYPE"] != 'Other Motorized']

In [16]:
print(drivers['CONTRIBUTING_FACTOR_1'].unique())
print(drivers['CONTRIBUTING_FACTOR_2'].unique())
print(drivers['PED_LOCATION'].unique())
print(drivers['PED_ACTION'].unique())
print(drivers['PERSON_TYPE'].unique())

[nan]
[nan]
[nan]
[nan]
['Occupant']


In [17]:
print(inVehiclePeople['CONTRIBUTING_FACTOR_1'].unique())
print(inVehiclePeople['CONTRIBUTING_FACTOR_2'].unique())
print(inVehiclePeople['PED_LOCATION'].unique())
print(inVehiclePeople['PED_ACTION'].unique())
print(inVehiclePeople['PERSON_TYPE'].unique())

[nan]
[nan]
[nan]
[nan]
['Occupant']


#### We know the data only has drivers, so we can remove the columsn that only have one value too and nan

In [18]:
drivers.drop(['CONTRIBUTING_FACTOR_1', 'CONTRIBUTING_FACTOR_2', 'PED_LOCATION', 'PED_ACTION', 'PED_ROLE', 'PERSON_TYPE', 'POSITION_IN_VEHICLE'], axis=1, inplace=True)
drivers

,COLLISION_ID,CRASH_TIME,PERSON_INJURY,VEHICLE_ID,PERSON_AGE,EJECTION,EMOTIONAL_STATUS,BODILY_INJURY,SAFETY_EQUIPMENT,COMPLAINT,PERSON_SEX,CRASH_DATE_YEAR,CRASH_DATE_MONTH,CRASH_DATE_DAY
5,4230743,19:15,Injured,19143343.0,27.0,Not Ejected,Conscious,Back,Lap Belt & Harness,Complaint of Pain or Nausea,M,2019,10,24
8,4229808,13:04,Unspecified,19141630.0,36.0,Not Ejected,Does Not Apply,Does Not Apply,Lap Belt & Harness,Does Not Apply,M,2019,10,26
10,4230737,0:41,Unspecified,19143334.0,30.0,Not Ejected,Does Not Apply,Does Not Apply,Lap Belt & Harness,Does Not Apply,M,2019,10,24
19,4230612,16:12,Unspecified,19143548.0,55.0,Not Ejected,Does Not Apply,Does Not Apply,Lap Belt & Harness,Does Not Apply,M,2019,10,24
27,4230765,9:55,Unspecified,19143969.0,59.0,Not Ejected,Does Not Apply,Does Not Apply,Lap Belt & Harness,Does Not Apply,M,2019,10,26
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5513725,4767221,7:54,Unspecified,20773432.0,59.0,Not Ejected,Does Not Apply,Does Not Apply,Unknown,Does Not Apply,F,2024,10,29
5513732,4767506,15:45,Unspecified,20773902.0,34.0,Not Ejected,Does Not Apply,Does Not Apply,Lap Belt,Does Not Apply,M,2024,10,29
5513733,4767480,15:15,Unspecified,20774286.0,23.0,Not Ejected,Does Not Apply,Does Not Apply,Child Restraint Only,Does Not Apply,M,2024,10,29
5513735,4767575,13:30,Unspecified,20774084.0,66.0,Not Ejected,Does Not Apply,Does Not Apply,Lap Belt & Harness,Does Not Apply,M,2024,10,29


In [19]:
inVehiclePeople.drop(['CONTRIBUTING_FACTOR_1', 'CONTRIBUTING_FACTOR_2', 'PED_LOCATION', 'PED_ACTION', 'PED_ROLE', 'PERSON_TYPE', 'POSITION_IN_VEHICLE'], axis=1, inplace=True)
inVehiclePeople

,COLLISION_ID,CRASH_TIME,PERSON_INJURY,VEHICLE_ID,PERSON_AGE,EJECTION,EMOTIONAL_STATUS,BODILY_INJURY,SAFETY_EQUIPMENT,COMPLAINT,PERSON_SEX,CRASH_DATE_YEAR,CRASH_DATE_MONTH,CRASH_DATE_DAY
1,4230587,15:15,Unspecified,19144075.0,33.0,Not Ejected,Does Not Apply,Does Not Apply,Lap Belt & Harness,Does Not Apply,F,2019,10,25
4,4231168,11:16,Unspecified,19144329.0,7.0,Not Ejected,Does Not Apply,Does Not Apply,Lap Belt,Does Not Apply,F,2019,10,25
5,4230743,19:15,Injured,19143343.0,27.0,Not Ejected,Conscious,Back,Lap Belt & Harness,Complaint of Pain or Nausea,M,2019,10,24
8,4229808,13:04,Unspecified,19141630.0,36.0,Not Ejected,Does Not Apply,Does Not Apply,Lap Belt & Harness,Does Not Apply,M,2019,10,26
10,4230737,0:41,Unspecified,19143334.0,30.0,Not Ejected,Does Not Apply,Does Not Apply,Lap Belt & Harness,Does Not Apply,M,2019,10,24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5513741,4767880,14:40,Injured,20774505.0,49.0,Not Ejected,Conscious,Knee-Lower Leg Foot,NaN,Complaint of Pain or Nausea,F,2024,10,28
5513744,4767904,15:32,Injured,20774759.0,71.0,Not Ejected,Unknown,Neck,Lap Belt & Harness,Complaint of Pain or Nausea,F,2024,10,29
5513745,4767864,17:00,Unspecified,20774731.0,49.0,Not Ejected,Does Not Apply,Does Not Apply,Lap Belt & Harness,Does Not Apply,F,2024,10,29
5513747,4767781,16:10,Unspecified,20774464.0,35.0,Not Ejected,Does Not Apply,Does Not Apply,Lap Belt & Harness,Does Not Apply,F,2024,10,24


In [20]:
drivers.drop(['CRASH_TIME', 'CRASH_DATE_YEAR', 'CRASH_DATE_MONTH', 'CRASH_DATE_DAY'], axis=1, inplace=True)
inVehiclePeople.drop(['CRASH_TIME', 'CRASH_DATE_YEAR', 'CRASH_DATE_MONTH', 'CRASH_DATE_DAY'], axis=1, inplace=True)

In [21]:
drivers.columns

Index(['COLLISION_ID', 'PERSON_INJURY', 'VEHICLE_ID', 'PERSON_AGE', 'EJECTION',
       'EMOTIONAL_STATUS', 'BODILY_INJURY', 'SAFETY_EQUIPMENT', 'COMPLAINT',
       'PERSON_SEX'],
      dtype='object')

In [22]:
inVehiclePeople.columns

Index(['COLLISION_ID', 'PERSON_INJURY', 'VEHICLE_ID', 'PERSON_AGE', 'EJECTION',
       'EMOTIONAL_STATUS', 'BODILY_INJURY', 'SAFETY_EQUIPMENT', 'COMPLAINT',
       'PERSON_SEX'],
      dtype='object')

In [23]:
ct = make_column_transformer(
(preprocessing.OneHotEncoder(sparse_output=False), 
['PERSON_INJURY', 'EJECTION', 'EMOTIONAL_STATUS', 'BODILY_INJURY', 'SAFETY_EQUIPMENT', 'COMPLAINT', 'PERSON_SEX']), 
remainder="passthrough")
ct.set_output(transform="pandas")

inVehiclePeople = ct.fit_transform(inVehiclePeople)
drivers = ct.fit_transform(drivers)

In [24]:
inVehiclePeople.columns = inVehiclePeople.columns.str.replace('onehotencoder__', '', regex=False)
inVehiclePeople.columns = inVehiclePeople.columns.str.replace('remainder__', '', regex=False)
inVehiclePeople

,PERSON_INJURY_Injured,PERSON_INJURY_Killed,PERSON_INJURY_Unspecified,EJECTION_Does Not Apply,EJECTION_Ejected,EJECTION_Not Ejected,EJECTION_Partially Ejected,EJECTION_Trapped,EJECTION_Unknown,EJECTION_nan,...,COMPLAINT_Severe Lacerations,COMPLAINT_Unknown,COMPLAINT_Whiplash,COMPLAINT_nan,PERSON_SEX_F,PERSON_SEX_M,PERSON_SEX_U,COLLISION_ID,VEHICLE_ID,PERSON_AGE
1,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,4230587,19144075.0,33.0
4,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,4231168,19144329.0,7.0
5,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,4230743,19143343.0,27.0
8,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,4229808,19141630.0,36.0
10,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,4230737,19143334.0,30.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5513741,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,4767880,20774505.0,49.0
5513744,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,4767904,20774759.0,71.0
5513745,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,4767864,20774731.0,49.0
5513747,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,4767781,20774464.0,35.0


In [25]:
drivers.columns = drivers.columns.str.replace('onehotencoder__', '', regex=False)
drivers.columns = drivers.columns.str.replace('remainder__', '', regex=False)
drivers

,PERSON_INJURY_Injured,PERSON_INJURY_Killed,PERSON_INJURY_Unspecified,EJECTION_Does Not Apply,EJECTION_Ejected,EJECTION_Not Ejected,EJECTION_Partially Ejected,EJECTION_Trapped,EJECTION_Unknown,EJECTION_nan,...,COMPLAINT_Severe Lacerations,COMPLAINT_Unknown,COMPLAINT_Whiplash,COMPLAINT_nan,PERSON_SEX_F,PERSON_SEX_M,PERSON_SEX_U,COLLISION_ID,VEHICLE_ID,PERSON_AGE
5,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,4230743,19143343.0,27.0
8,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,4229808,19141630.0,36.0
10,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,4230737,19143334.0,30.0
19,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,4230612,19143548.0,55.0
27,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,4230765,19143969.0,59.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5513725,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,4767221,20773432.0,59.0
5513732,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,4767506,20773902.0,34.0
5513733,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,4767480,20774286.0,23.0
5513735,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,4767575,20774084.0,66.0


In [26]:
drivers.to_csv('../data/processed/drivers_processed.csv', index=False) 
inVehiclePeople.to_csv('../data/processed/invehicle_processed.csv', index=False) 

In [ ]:
# Cyclic Encoding
# Extract the hour part only
# drivers['CRASH_HOUR'] = drivers['CRASH_TIME'].str.split(':').str[0].astype(int)

# # Transform month, day, and hour into sine and cosine components
# drivers['month_sin'] = np.sin(2 * np.pi * drivers['CRASH_DATE_MONTH'] / 12)
# drivers['month_cos'] = np.cos(2 * np.pi * drivers['CRASH_DATE_MONTH'] / 12)

# drivers['day_sin'] = np.sin(2 * np.pi * drivers['CRASH_DATE_DAY'] / 31)
# drivers['day_cos'] = np.cos(2 * np.pi * drivers['CRASH_DATE_DAY'] / 31)

# drivers['hour_sin'] = np.sin(2 * np.pi * drivers['CRASH_HOUR'] / 24)
# drivers['hour_cos'] = np.cos(2 * np.pi * drivers['CRASH_HOUR'] / 24)

# # Drop the original columns if desired
# # Dropping year because do not want that to affect the data
# drivers = drivers.drop(['CRASH_DATE_MONTH', 'CRASH_DATE_DAY', 'CRASH_HOUR', 'CRASH_TIME', 'CRASH_DATE_YEAR'], axis=1)

KeyError: 'CRASH_TIME'

In [ ]:
# inVehiclePeople['CRASH_HOUR'] = inVehiclePeople['CRASH_TIME'].str.split(':').str[0].astype(int)


# # Transform month, day, and hour into sine and cosine components
# inVehiclePeople['month_sin'] = np.sin(2 * np.pi * inVehiclePeople['CRASH_DATE_MONTH'] / 12)
# inVehiclePeople['month_cos'] = np.cos(2 * np.pi * inVehiclePeople['CRASH_DATE_MONTH'] / 12)

# inVehiclePeople['day_sin'] = np.sin(2 * np.pi * inVehiclePeople['CRASH_DATE_DAY'] / 31)
# inVehiclePeople['day_cos'] = np.cos(2 * np.pi * inVehiclePeople['CRASH_DATE_DAY'] / 31)

# inVehiclePeople['hour_sin'] = np.sin(2 * np.pi * inVehiclePeople['CRASH_HOUR'] / 24)
# inVehiclePeople['hour_cos'] = np.cos(2 * np.pi * inVehiclePeople['CRASH_HOUR'] / 24)

# # Drop the original columns if desired
# # Dropping year because do not want that to affect the data
# inVehiclePeople = inVehiclePeople.drop(['CRASH_DATE_MONTH', 'CRASH_DATE_DAY', 'CRASH_HOUR', 'CRASH_TIME', 'CRASH_DATE_YEAR'], axis=1)